In [86]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas

import pandas as pd
# this helps us preview more rows and columns in our dataset
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# this prevents pandas from displaying floats with scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import re
import numpy as np

# read in CSV files of benchmarking data
data2018 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018_.csv")
data2017 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_.csv")
data2016 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.csv")
data2015 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_.csv")
data2014 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_.csv")
data2013 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_.csv")
data2012 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_.csv")
data2011 = pd.read_csv("data/LL84_benchmark/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2012__Data_for_Calendar_Year_2011_.csv")

In [87]:
#transform the csvs so we have fewer empty rows and all columns have consistent names
data2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25939 entries, 0 to 25938
Data columns (total 67 columns):
Order                                                         25939 non-null int64
Property Id                                                   25939 non-null int64
Property Name                                                 25939 non-null object
Parent Property Id                                            25939 non-null object
Parent Property Name                                          25939 non-null object
City Building                                                 25939 non-null object
BBL - 10 digits                                               25939 non-null object
NYC Borough, Block and Lot (BBL) self - reported              25939 non-null object
NYC Building Identification Number (BIN)                      25939 non-null object
Address 1 (self - reported)                                   25939 non-null object
Address 2 (self - reported)                          

In [195]:
s = data2018['BBL - 10 digits'].str.split(r'([,;:&-]|and)').apply(pd.Series, 1).stack()
s.str.strip(r'(\s)')
s.index = s.index.droplevel(-1)
s.name = 'BBL'

newBBLs2018 = data2018.join(s)
newBBLs2018.reset_index(drop=True, inplace=True)

filtered2018 = newBBLs2018[newBBLs2018['BBL'].str.match(r'(\d{10})') == True]
filtered2018['BBL'] = filtered2018['BBL'].astype(int)
# filtered2018.head()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [196]:
# drop columns we won't visualize to reduce file size
filtered2018.drop(['BBL - 10 digits', 'Property Id', 'Parent Property Id', 'Parent Property Name', 'City Building','NYC Building Identification Number (BIN)', 'NYC Borough, Block and Lot (BBL) self - reported', 'Address 2 (self - reported)', 'Postcode', 'Street Number', 'Street Name', 'DOF Gross Floor Area  (ft²)', 'List of All Property Use Types at Property', 'Largest Property Use Type', 'Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', '2nd Largest Property Use - Gross Floor Area (ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Number of Buildings', 'Occupancy', 'Metered Areas (Energy)', 'Metered Areas  (Water)', 'Fuel Oil #1 Use (kBtu)', 'Fuel Oil #2 Use (kBtu)', 'Fuel Oil #4 Use (kBtu)', 'Fuel Oil #5 & 6 Use (kBtu)', 'Diesel #2 Use (kBtu)', 'Kerosene Use (kBtu)', 'Propane Use (kBtu)', 'Natural Gas Use (kBtu)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Site Natural Gas Use (therms)',  'District Steam Use (kBtu)', 'District Hot Water Use (kBtu)', 'District Chilled Water Use (kBtu)', 'Annual Maximum Demand (kW)', 'Annual Maximum Demand (MM/YYYY)', 'Water Use (All Water Sources) (kgal)', 'Water Use Intensity (All Water Sources) (gal/ft²)', 'Water Required?', 'Generation Date', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'NTA'], axis = 1, inplace = True)
filtered2018.head()

,Order,Property Name,Address 1 (self - reported),Borough,Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL
0,1,1155,1155 Avenue of the Americas,Manhattan,686838,Office,1982,79,110.9,110.9,45.7,45.8,10.3,2.39953415E7,7032631.5,7032631.5,2520.9,0,2520.9,1009970029
1,2,200,200 East 42nd St.,Manhattan,384432,Office,1956,62,193.4,190.6,101,98.9,13.2,1.73088981E7,5072947.2,5055495,2895.1,0,2895.1,1013150001
2,3,114,114 West 47th st,Manhattan,577534,Financial Office,1989,66,155.6,154.6,70.3,70.3,12.9,2.57830572E7,7556580.9,7446568.4,3155.8,51.2,3104.7,1009990019
3,4,733,733 Third Avenue,Manhattan,390878,Office,1961,64,181.5,178.2,97.1,95,11.7,1.59062232E7,4661846.8,4612102.7,2826.2,0,2826.2,1013190047
4,5,Conde Nast Building,4 Times Square,Manhattan,1601977,Office,1999,42,226.1,227.3,100.4,101.5,20,1.092587034E8,3.20218904E7,3.20218904E7,12179.3,1991.7,10187.6,1009950005


In [197]:
# convert columns to floats or ints if that's what they contain
# filtered2018['Year Built'].value_counts()
cols_to_number = ['Self-Reported Gross Floor Area (ft²)', 'Year Built', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Electricity Use - Grid Purchase (kBtu)', 'Electricity Use - Grid Purchase (kWh)', 'Weather Normalized Site Electricity (kWh)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Indirect GHG Emissions (Metric Tons CO2e)' ]
filtered2018[cols_to_number] = filtered2018[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2018.describe()


,Order,Self-Reported Gross Floor Area (ft²),Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL
count,26080.00000,26080.00000,26080.00000,21044.00000,25674.00000,23855.00000,25674.00000,23855.00000,25149.00000,25531.00000,25531.00000,25150.00000,25874.00000,25875.00000,25932.00000,26080.00000
mean,12938.91967,147334.11894,1947.29816,52.20628,154.28970,155.26174,99.34351,100.31445,8.12158,4697271.24183,1376691.28834,1371924.38584,1297.32508,535.42590,760.42229,2622388992.27274
std,7473.39065,539874.46833,33.93344,30.46740,1284.34604,1343.70634,1065.28048,1114.60868,9.42000,17785385.78963,5212597.78083,5225147.52066,43687.48214,3862.04340,43391.68189,31488603430.65096
min,1.00000,200.00000,1051.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-21909.70000,1000020001.00000
25%,6491.75000,39656.50000,1925.00000,27.00000,98.60000,98.80000,63.10000,63.30000,3.90000,705293.85000,206709.80000,202485.05000,202.30000,82.40000,58.00000,1014917501.75000
50%,12907.50000,63361.00000,1936.00000,53.00000,126.50000,126.70000,84.30000,84.80000,5.40000,1271443.00000,372638.60000,366145.85000,343.50000,201.80000,106.00000,2032560153.00000
75%,19392.25000,115142.50000,1967.00000,78.00000,163.07500,163.40000,108.00000,108.60000,9.50000,2942145.10000,862293.30000,848283.60000,626.37500,350.45000,253.00000,3058912543.75000
max,25939.00000,21150159.00000,2088.00000,100.00000,200466.60000,202085.80000,166498.40000,167844.30000,440.60000,593424464.10000,173922740.90000,173566867.90000,6965938.70000,144497.30000,6965792.00000,4202038640001.00000


In [198]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2018['Emissions Per Sq Ft'] = np.where(filtered2018['Self-Reported Gross Floor Area (ft²)'] <= 0, 0, filtered2018['Total GHG Emissions (Metric Tons CO2e)'] / filtered2018['Self-Reported Gross Floor Area (ft²)'])
filtered2018.describe()


/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Order,Self-Reported Gross Floor Area (ft²),Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL,Emissions Per Sq Ft
count,26080.00000,26080.00000,26080.00000,21044.00000,25674.00000,23855.00000,25674.00000,23855.00000,25149.00000,25531.00000,25531.00000,25150.00000,25874.00000,25875.00000,25932.00000,26080.00000,25874.00000
mean,12938.91967,147334.11894,1947.29816,52.20628,154.28970,155.26174,99.34351,100.31445,8.12158,4697271.24183,1376691.28834,1371924.38584,1297.32508,535.42590,760.42229,2622388992.27274,0.00648
std,7473.39065,539874.46833,33.93344,30.46740,1284.34604,1343.70634,1065.28048,1114.60868,9.42000,17785385.78963,5212597.78083,5225147.52066,43687.48214,3862.04340,43391.68189,31488603430.65096,0.07046
min,1.00000,200.00000,1051.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-21909.70000,1000020001.00000,0.00000
25%,6491.75000,39656.50000,1925.00000,27.00000,98.60000,98.80000,63.10000,63.30000,3.90000,705293.85000,206709.80000,202485.05000,202.30000,82.40000,58.00000,1014917501.75000,0.00410
50%,12907.50000,63361.00000,1936.00000,53.00000,126.50000,126.70000,84.30000,84.80000,5.40000,1271443.00000,372638.60000,366145.85000,343.50000,201.80000,106.00000,2032560153.00000,0.00540
75%,19392.25000,115142.50000,1967.00000,78.00000,163.07500,163.40000,108.00000,108.60000,9.50000,2942145.10000,862293.30000,848283.60000,626.37500,350.45000,253.00000,3058912543.75000,0.00697
max,25939.00000,21150159.00000,2088.00000,100.00000,200466.60000,202085.80000,166498.40000,167844.30000,440.60000,593424464.10000,173922740.90000,173566867.90000,6965938.70000,144497.30000,6965792.00000,4202038640001.00000,11.05705


In [199]:
# rename some of the columns to compare datasets across years more easily
filtered2018.rename(columns={'Order': 'order',
                             'Property Name': 'prop_name',
                             'Address 1 (self - reported)': 'address',
                             'Borough': 'borough',
                             'Self-Reported Gross Floor Area (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI (kBtu/ft²)': 'src_eui',
                             'Weather Normalized Source EUI (kBtu/ft²)': 'wnsrc_eui',
                             'Site EUI (kBtu/ft²)': 'site_eui',
                             'Weather Normalized Site EUI (kBtu/ft²)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions (Metric Tons CO2e)': 'total_ghg',
                             'Direct GHG Emissions (Metric Tons CO2e)': 'direct_ghg',
                             'Indirect GHG Emissions (Metric Tons CO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)
filtered2018.head()

,order,prop_name,address,borough,sq_footage,prop_type,year_built,energ_star,src_eui,wnsrc_eui,site_eui,wnsite_eui,wn_sei_kwh,eu_gp_kbtu,eu_gp_kwh,wnse_kwh,total_ghg,direct_ghg,indrct_ghg,bbl,t_ghg_psqf
0,1,1155,1155 Avenue of the Americas,Manhattan,686838,Office,1982,79.00000,110.90000,110.90000,45.70000,45.80000,10.30000,23995341.50000,7032631.50000,7032631.50000,2520.90000,0.00000,2520.90000,1009970029,0.00367
1,2,200,200 East 42nd St.,Manhattan,384432,Office,1956,62.00000,193.40000,190.60000,101.00000,98.90000,13.20000,17308898.10000,5072947.20000,5055495.00000,2895.10000,0.00000,2895.10000,1013150001,0.00753
2,3,114,114 West 47th st,Manhattan,577534,Financial Office,1989,66.00000,155.60000,154.60000,70.30000,70.30000,12.90000,25783057.20000,7556580.90000,7446568.40000,3155.80000,51.20000,3104.70000,1009990019,0.00546
3,4,733,733 Third Avenue,Manhattan,390878,Office,1961,64.00000,181.50000,178.20000,97.10000,95.00000,11.70000,15906223.20000,4661846.80000,4612102.70000,2826.20000,0.00000,2826.20000,1013190047,0.00723
4,5,Conde Nast Building,4 Times Square,Manhattan,1601977,Office,1999,42.00000,226.10000,227.30000,100.40000,101.50000,20.00000,109258703.40000,32021890.40000,32021890.40000,12179.30000,1991.70000,10187.60000,1009950005,0.00760


In [7]:
data2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 66 columns):
Order                                                         34686 non-null int64
Property Id                                                   34686 non-null int64
Property Name                                                 34686 non-null object
Parent Property Id                                            1980 non-null float64
Parent Property Name                                          1980 non-null object
BBL - 10 digits                                               33956 non-null object
NYC Borough, Block and Lot (BBL) self-reported                33957 non-null object
NYC Building Identification Number (BIN)                      33426 non-null object
Address 1 (self-reported)                                     34686 non-null object
Address 2 (self-reported)                                     462 non-null object
Postal Code                                             

In [215]:
s = data2017['BBL - 10 digits'].str.split(r'([,;:&-]|and)').apply(pd.Series, 1).stack()
s.str.strip(r'(\s)')
s.index = s.index.droplevel(-1)
s.name = 'BBL'

newBBLs2017 = data2017.join(s)
newBBLs2017.reset_index(drop=True, inplace=True)

filtered2017 = newBBLs2017[newBBLs2017['BBL'].str.match(r'(\d{10})') == True]
filtered2017['BBL'] = filtered2017['BBL'].astype(int)
filtered2017.head()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),Postal Code,Street Number,Street Name,Borough,DOF Gross Floor Area (ft²),Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²),Water Required?,Generation Date,DOF Benchmarking Submission Status,Latitude,Longitude,Community Board,Council District,Census Tract,NTA,BBL
0,1,4593574,The Argonaut Building,nan,NaN,1010287502,1010287502,1024898,224 West 57th St,NaN,10019,Not found,Not found,Manhattan,Not found,169416,Office,"Bank Branch, Office",Office,164754.00000,Bank Branch,4662.00000,NaN,nan,1909,1,95,Whole Building,Whole Building,90.00000,138.40000,141.00000,53.80000,56.20000,11.30000,0.10000,nan,nan,nan,nan,nan,nan,1133475.10000,nan,nan,1435754.70000,16672.20000,6551394.10000,1920103.60000,1920103.60000,nan,NaN,732.40000,76.30000,656.10000,3635.50000,21.46000,Not found,02/14/2018,Not found,40.76595,-73.98107,105.00000,4.00000,137.00000,Midtown-Midtown South,1010287502
1,2,6224375,Operative Cakes,nan,NaN,2005600062;2055600062,2-00560-0062;2-05560-0062,2087721;2080293,711-733 Brush Avenue,NaN,10465,Not found,Not found,Bronx,Not found,104407,Non-Refrigerated Warehouse,Non-Refrigerated Warehouse,Non-Refrigerated Warehouse,104407.00000,NaN,nan,NaN,nan,1973,2,100,Whole Building,Whole Building,nan,84.60000,nan,32.00000,nan,7.10000,nan,nan,773283.00000,nan,nan,nan,nan,nan,nan,nan,nan,nan,2564587.10000,751637.40000,745273.80000,nan,NaN,284.80000,57.40000,227.40000,116.00000,1.11000,Not found,02/14/2018,Not found,40.82121,-73.83789,210.00000,13.00000,90.00000,Soundview-Castle Hill-Clason Point-Harding Park,2005600062
3,2,6224375,Operative Cakes,nan,NaN,2005600062;2055600062,2-00560-0062;2-05560-0062,2087721;2080293,711-733 Brush Avenue,NaN,10465,Not found,Not found,Bronx,Not found,104407,Non-Refrigerated Warehouse,Non-Refrigerated Warehouse,Non-Refrigerated Warehouse,104407.00000,NaN,nan,NaN,nan,1973,2,100,Whole Building,Whole Building,nan,84.60000,nan,32.00000,nan,7.10000,nan,nan,773283.00000,nan,nan,nan,nan,nan,nan,nan,nan,nan,2564587.10000,751637.40000,745273.80000,nan,NaN,284.80000,57.40000,227.40000,116.00000,1.11000,Not found,02/14/2018,Not found,40.82121,-73.83789,210.00000,13.00000,90.00000,Soundview-Castle Hill-Clason Point-Harding Park,2055600062
4,3,2967701,Cathedral Preparatory Seminary,nan,NaN,4018720007,4-01872-0007,4046340,56-25 92nd Street,NaN,11373,Not found,Not found,Queens,Not found,94380,K-12 School,K-12 School,K-12 School,94380.00000,NaN,nan,NaN,nan,1963,1,100,Whole Building,NaN,100.000

In [216]:
# drop columns we won't visualize to reduce file size
filtered2017.drop(['BBL - 10 digits', 'Property Id', 'Parent Property Id', 'Parent Property Name', 'NYC Borough, Block and Lot (BBL) self-reported', 'NYC Building Identification Number (BIN)', 'Address 2 (self-reported)', 'Postal Code', 'Street Number', 'Street Name', 'DOF Gross Floor Area (ft²)', 'List of All Property Use Types at Property', 'Largest Property Use Type', 'Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', '2nd Largest Property Use - Gross Floor Area (ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Number of Buildings', 'Occupancy', 'Metered Areas (Energy)', 'Metered Areas  (Water)', 'Fuel Oil #1 Use (kBtu)', 'Fuel Oil #2 Use (kBtu)', 'Fuel Oil #4 Use (kBtu)', 'Fuel Oil #5 & 6 Use (kBtu)', 'Diesel #2 Use (kBtu)', 'Propane Use (kBtu)', 'Natural Gas Use (kBtu)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Site Natural Gas Use (therms)', 'District Steam Use (kBtu)', 'District Hot Water Use (kBtu)', 'District Chilled Water Use (kBtu)', 'Annual Maximum Demand (kW)', 'Annual Maximum Demand (MM/YYYY)', 'Water Use (All Water Sources) (kgal)', 'Water Use Intensity (All Water Sources) (gal/ft²)', 'Water Required?', 'Generation Date','DOF Benchmarking Submission Status', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'NTA'], axis = 1, inplace = True)
filtered2017.head()

,Order,Property Name,Address 1 (self-reported),Borough,Self-Reported Gross Floor Area (ft²),Primary Property Type - Self Selected,Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL
0,1,The Argonaut Building,224 West 57th St,Manhattan,169416,Office,1909,90.00000,138.40000,141.00000,53.80000,56.20000,11.30000,6551394.10000,1920103.60000,1920103.60000,732.40000,76.30000,656.10000,1010287502
1,2,Operative Cakes,711-733 Brush Avenue,Bronx,104407,Non-Refrigerated Warehouse,1973,nan,84.60000,nan,32.00000,nan,7.10000,2564587.10000,751637.40000,745273.80000,284.80000,57.40000,227.40000,2005600062
3,2,Operative Cakes,711-733 Brush Avenue,Bronx,104407,Non-Refrigerated Warehouse,1973,nan,84.60000,nan,32.00000,nan,7.10000,2564587.10000,751637.40000,745273.80000,284.80000,57.40000,227.40000,2055600062
4,3,Cathedral Preparatory Seminary,56-25 92nd Street,Queens,94380,K-12 School,1963,100.00000,43.50000,46.80000,28.40000,31.30000,2.00000,616343.70000,180640.00000,184131.90000,164.50000,109.90000,54.60000,4018720007
5,4,The Nomad Hotel,1170 Broadway,Manhattan,125000,Hotel,1999,83.00000,271.10000,274.70000,130.20000,133.70000,18.80000,8033914.40000,2354605.30000,2354605.30000,1150.20000,438.00000,712.30000,1008290050


In [217]:
# convert columns to floats or ints if that's what they contain
# filtered2018['Year Built'].value_counts()
cols_to_number = ['Self-Reported Gross Floor Area (ft²)', 'Year Built', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Electricity Use - Grid Purchase (kBtu)', 'Electricity Use - Grid Purchase (kWh)', 'Weather Normalized Site Electricity (kWh)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Indirect GHG Emissions (Metric Tons CO2e)' ]
filtered2017[cols_to_number] = filtered2017[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2017.describe()


,Order,Self-Reported Gross Floor Area (ft²),Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL
count,34798.00000,34798.00000,34798.00000,25518.00000,32415.00000,29034.00000,32415.00000,29034.00000,31461.00000,32052.00000,32052.00000,31470.00000,33440.00000,33610.00000,33586.00000,34798.00000
mean,17322.50060,228554.60437,1945.29867,53.81221,1147.98688,1258.58311,924.59909,1016.43528,8.36349,6563991.17160,1923795.53764,1900606.51766,23048.36514,5765.77204,17182.38468,2246556916.66989
std,9946.98817,1018388.30594,34.03932,32.53398,160803.42363,169916.51620,133666.46252,141241.57311,11.74928,27063969.34043,7931994.73918,7762608.43913,2933134.55590,169439.90910,2921852.01727,1198567461.62501
min,1.00000,0.00000,1051.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-6343.30000,-1859.10000,-1859.10000,0.00000,0.00000,-17600.50000,1000020001.00000
25%,8803.25000,43839.00000,1924.00000,25.00000,93.90000,97.80000,58.40000,61.10000,3.80000,711437.12500,208510.30000,208988.55000,192.20000,66.90000,55.50000,1014960021.00000
50%,17277.50000,68640.00000,1935.00000,56.00000,123.30000,128.40000,80.40000,84.40000,5.30000,1315182.50000,385457.90000,387136.50000,342.95000,197.00000,107.20000,2032470001.00000
75%,25900.75000,122742.25000,1965.00000,83.00000,165.65000,171.60000,104.70000,109.80000,9.50000,3151934.10000,923778.90000,928439.42500,646.40000,350.90000,267.00000,3056560004.75000
max,34686.00000,21150159.00000,2021.00000,100.00000,28942940.00000,28942950.00000,24058060.00000,24058060.00000,449.60000,574283382.30000,168312811.40000,167695514.00000,535429700.00000,20833880.00000,535429700.00000,30605000019.00000


In [218]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2017['Emissions Per Sq Ft'] = np.where(filtered2017['Self-Reported Gross Floor Area (ft²)'] <= 0, 0, filtered2017['Total GHG Emissions (Metric Tons CO2e)'] / filtered2017['Self-Reported Gross Floor Area (ft²)'])
filtered2017.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Order,Self-Reported Gross Floor Area (ft²),Year Built,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),BBL,Emissions Per Sq Ft
count,34798.00000,34798.00000,34798.00000,25518.00000,32415.00000,29034.00000,32415.00000,29034.00000,31461.00000,32052.00000,32052.00000,31470.00000,33440.00000,33610.00000,33586.00000,34798.00000,33440.00000
mean,17322.50060,228554.60437,1945.29867,53.81221,1147.98688,1258.58311,924.59909,1016.43528,8.36349,6563991.17160,1923795.53764,1900606.51766,23048.36514,5765.77204,17182.38468,2246556916.66989,0.18468
std,9946.98817,1018388.30594,34.03932,32.53398,160803.42363,169916.51620,133666.46252,141241.57311,11.74928,27063969.34043,7931994.73918,7762608.43913,2933134.55590,169439.90910,2921852.01727,1198567461.62501,15.86147
min,1.00000,0.00000,1051.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-6343.30000,-1859.10000,-1859.10000,0.00000,0.00000,-17600.50000,1000020001.00000,0.00000
25%,8803.25000,43839.00000,1924.00000,25.00000,93.90000,97.80000,58.40000,61.10000,3.80000,711437.12500,208510.30000,208988.55000,192.20000,66.90000,55.50000,1014960021.00000,0.00371
50%,17277.50000,68640.00000,1935.00000,56.00000,123.30000,128.40000,80.40000,84.40000,5.30000,1315182.50000,385457.90000,387136.50000,342.95000,197.00000,107.20000,2032470001.00000,0.00513
75%,25900.75000,122742.25000,1965.00000,83.00000,165.65000,171.60000,104.70000,109.80000,9.50000,3151934.10000,923778.90000,928439.42500,646.40000,350.90000,267.00000,3056560004.75000,0.00686
max,34686.00000,21150159.00000,2021.00000,100.00000,28942940.00000,28942950.00000,24058060.00000,24058060.00000,449.60000,574283382.30000,168312811.40000,167695514.00000,535429700.00000,20833880.00000,535429700.00000,30605000019.00000,1597.42022


In [219]:
# rename some of the columns to compare datasets across years more easily
filtered2017.rename(columns={'Order': 'order',
                             'Property Name': 'prop_name',
                             'Address 1 (self-reported)': 'address',
                             'Borough': 'borough',
                             'Self-Reported Gross Floor Area (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI (kBtu/ft²)': 'src_eui',
                             'Weather Normalized Source EUI (kBtu/ft²)': 'wnsrc_eui',
                             'Site EUI (kBtu/ft²)': 'site_eui',
                             'Weather Normalized Site EUI (kBtu/ft²)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions (Metric Tons CO2e)': 'total_ghg',
                             'Direct GHG Emissions (Metric Tons CO2e)': 'direct_ghg',
                             'Indirect GHG Emissions (Metric Tons CO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)
filtered2017.head()

,order,prop_name,address,borough,sq_footage,prop_type,year_built,energ_star,src_eui,wnsrc_eui,site_eui,wnsite_eui,wn_sei_kwh,eu_gp_kbtu,eu_gp_kwh,wnse_kwh,total_ghg,direct_ghg,indrct_ghg,bbl,t_ghg_psqf
0,1,The Argonaut Building,224 West 57th St,Manhattan,169416,Office,1909,90.00000,138.40000,141.00000,53.80000,56.20000,11.30000,6551394.10000,1920103.60000,1920103.60000,732.40000,76.30000,656.10000,1010287502,0.00432
1,2,Operative Cakes,711-733 Brush Avenue,Bronx,104407,Non-Refrigerated Warehouse,1973,nan,84.60000,nan,32.00000,nan,7.10000,2564587.10000,751637.40000,745273.80000,284.80000,57.40000,227.40000,2005600062,0.00273
3,2,Operative Cakes,711-733 Brush Avenue,Bronx,104407,Non-Refrigerated Warehouse,1973,nan,84.60000,nan,32.00000,nan,7.10000,2564587.10000,751637.40000,745273.80000,284.80000,57.40000,227.40000,2055600062,0.00273
4,3,Cathedral Preparatory Seminary,56-25 92nd Street,Queens,94380,K-12 School,1963,100.00000,43.50000,46.80000,28.40000,31.30000,2.00000,616343.70000,180640.00000,184131.90000,164.50000,109.90000,54.60000,4018720007,0.00174
5,4,The Nomad Hotel,1170 Broadway,Manhattan,125000,Hotel,1999,83.00000,271.10000,274.70000,130.20000,133.70000,18.80000,8033914.40000,2354605.30000,2354605.30000,1150.20000,438.00000,712.30000,1008290050,0.00920


In [264]:
data2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11746 entries, 0 to 11745
Data columns (total 61 columns):
Order                                                         11746 non-null int64
Property Id                                                   11746 non-null int64
Property Name                                                 11746 non-null object
Parent Property Id                                            11746 non-null object
Parent Property Name                                          11746 non-null object
BBL - 10 digits                                               11695 non-null float64
NYC Borough, Block and Lot (BBL) self-reported                11746 non-null object
NYC Building Identification Number (BIN)                      11746 non-null object
Address 1 (self-reported)                                     11746 non-null object
Address 2                                                     11746 non-null object
Postal Code                                         

In [268]:
# In 2016, our BBL data is already a non-string object so we don't need to do all the transformations
# we did for 2017 and 2018
data2016['BBL'] = data2016['BBL - 10 digits']
filtered2016 = data2016[data2016['BBL'].notnull()]
filtered2016.describe()

,Order,Property Id,BBL - 10 digits,Street Number,DOF Gross Floor Area,Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,Site EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Source EUI (kBtu/ft²),Latitude,Longitude,Community Board,Council District,Census Tract,BBL
count,11695.00000,11695.00000,11695.00000,9560.00000,11628.00000,11695.00000,11695.00000,11695.00000,11695.00000,11538.00000,11459.00000,11623.00000,11614.00000,11633.00000,11695.00000,11538.00000,9483.00000,9483.00000,9483.00000,9483.00000,9483.00000,11695.00000
mean,7210.93955,3638684.77110,2178124473.71928,1007.08734,173269.45442,156938.14371,1948.75648,1.27037,98.76828,280.74348,5809470.36103,4536.51638,2474.06794,2062.89925,163508.92065,386.65805,40.75438,-73.95706,7.14067,15.77127,4977.59665,2178124473.71928
std,4308.45665,1047295.32434,1222326211.19575,1577.52941,336705.45459,282515.22040,30.53021,3.92423,7.50595,8623.94060,30967891.53918,204590.94970,195861.05586,59429.02405,291282.07463,9330.86855,0.08012,0.04634,3.95413,15.67437,13520.42299,1222326211.19575
min,1.00000,7365.00000,1000047501.00000,1.00000,50028.00000,54.00000,1600.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-23134.30000,0.00000,0.00000,40.51607,-74.24358,1.00000,1.00000,1.00000,1000047501.00000
25%,3464.50000,2746919.00000,1012930048.00000,150.00000,65240.00000,65107.50000,1927.00000,1.00000,100.00000,61.80000,1043434.65000,328.50000,147.70000,94.70000,66926.50000,99.40000,40.70723,-73.98466,4.00000,4.00000,100.00000,1012930048.00000
50%,7020.00000,3211729.00000,2028770166.00000,446.00000,93138.50000,91000.00000,1941.00000,1.00000,100.00000,78.50000,1848203.90000,498.70000,272.25000,171.30000,93725.00000,124.80000,40.75913,-73.96281,7.00000,9.00000,201.00000,2028770166.00000
75%,11065.50000,4408868.50000,3064148753.50000,1304.50000,159614.00000,152107.00000,1966.00000,1.00000,100.00000,97.50000,4339162.35000,902.00000,446.57500,422.10000,157481.00000,162.50000,40.81762,-73.93244,9.00000,33.00000,531.50000,3064148753.50000
max,14993.00000,5991312.00000,5079910100.00000,76179.00000,13540113.00000,14217119.00000,2019.00000,161.00000,100.00000,869265.00000,1691763071.00000,20943400.00000,20943400.00000,4764375.00000,14217119.00000,912801.10000,40.91287,-73.71554,56.00000,51.00000,155101.00000,5079910100.00000


In [269]:
# drop columns we won't visualize to reduce file size
filtered2016.drop(['BBL - 10 digits', 'Property Id', 'Parent Property Id', 'Parent Property Name', 'NYC Borough, Block and Lot (BBL) self-reported', 'NYC Building Identification Number (BIN)', 'Address 2', 'Postal Code', 'DOF Gross Floor Area', 'List of All Property Use Types at Property', 'Largest Property Use Type', 'Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', '2nd Largest Property Use - Gross Floor Area (ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Occupancy', 'Metered Areas (Energy)', 'Metered Areas  (Water)', 'Fuel Oil #1 Use (kBtu)', 'Fuel Oil #2 Use (kBtu)', 'Fuel Oil #4 Use (kBtu)', 'Fuel Oil #5 & 6 Use (kBtu)', 'Diesel #2 Use (kBtu)', 'Natural Gas Use (kBtu)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Site Natural Gas Use (therms)', 'District Steam Use (kBtu)', 'Water Use (All Water Sources) (kgal)', 'Water Required?', 'DOF Benchmarking Submission Status', 'Community Board', 'Council District', 'Census Tract', 'NTA', 'Release Date', 'Water Intensity (All Water Sources) (gal/ft²)', 'Number of Buildings - Self-reported'], axis = 1, inplace = True)
filtered2016.head()

,Order,Property Name,Address 1 (self-reported),Street Number,Street Name,Borough,Primary Property Type - Self Selected,Year Built,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Source EUI (kBtu/ft²),Latitude,Longitude,BBL
0,1,201/205,201/205 East 42nd st.,675.00000,3 AVENUE,Manhattan,Office,1963,Not Available,305.60000,303.1,37.8,614.2,38139374.20000,1.10827705E7,6962.20000,0.00000,6962.20000,762051,619.40000,40.75079,-73.97396,1013160001.00000
1,2,NYP Columbia (West Campus),622 168th Street,180.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1969,55,229.80000,228.8,24.8,401.1,332365924.00000,9.62613121E7,55870.40000,51016.40000,4854.10000,3889181,404.30000,40.84140,-73.94257,1021380040.00000
2,3,MSCHoNY North,3975 Broadway,3975.00000,BROADWAY,Manhattan,Hospital (General Medical & Surgical),1924,Not Available,nan,Not Available,Not Available,Not Available,nan,Not Available,0.00000,0.00000,0.00000,231342,nan,40.84043,-73.94025,1021380030.00000
3,4,Herbert Irving Pavilion & Millstein Hospital,161 Fort Washington Ave,161.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1971,Not Available,nan,Not Available,Not Available,Not Available,nan,Not Available,0.00000,0.00000,0.00000,1305748,nan,40.84075,-73.94285,1021390001.00000
4,5,Neuro Institute,710 West 168th Street,193.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1932,Not Available,nan,Not Available,Not Available,Not Available,nan,Not Available,0.00000,0.00000,0.00000,179694,nan,40.84156,-73.94253,1021390085.00000


In [270]:
# pd.to_numeric(arg, errors=’raise’, downcast=None)
# convert columns to floats or ints if that's what they contain
# filtered2016['Year Built'].value_counts()
cols_to_number = ['Property GFA - Self-Reported (ft²)', 'Year Built', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Electricity Use - Grid Purchase (kBtu)', 'Weather Normalized Site Electricity (kWh)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Indirect GHG Emissions (Metric Tons CO2e)', 'Latitude', 'Longitude' ]
filtered2016[cols_to_number] = filtered2016[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2016['BBL'] = filtered2016['BBL'].astype(int)
filtered2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11695 entries, 0 to 11745
Data columns (total 23 columns):
Order                                                      11695 non-null int64
Property Name                                              11695 non-null object
Address 1 (self-reported)                                  11695 non-null object
Street Number                                              9560 non-null float64
Street Name                                                11624 non-null object
Borough                                                    11628 non-null object
Primary Property Type - Self Selected                      11695 non-null object
Year Built                                                 11695 non-null int64
ENERGY STAR Score                                          9618 non-null float64
Site EUI (kBtu/ft²)                                        11538 non-null float64
Weather Normalized Site EUI (kBtu/ft²)                     10242 non-null float64


/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [271]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2016['Emissions Per Sq Ft'] = np.where(filtered2016['Property GFA - Self-Reported (ft²)'] <= 0, 0, filtered2016['Total GHG Emissions (Metric Tons CO2e)'] /filtered2016['Property GFA - Self-Reported (ft²)'])
filtered2016.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Order,Street Number,Year Built,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Source EUI (kBtu/ft²),Latitude,Longitude,BBL,Emissions Per Sq Ft
count,11695.00000,9560.00000,11695.00000,9618.00000,11538.00000,10242.00000,10918.00000,10242.00000,11459.00000,10918.00000,11623.00000,11614.00000,11633.00000,11695.00000,11538.00000,9483.00000,9483.00000,11695.00000,11623.00000
mean,7210.93955,1007.08734,1948.75648,59.88480,280.74348,310.51127,11.06964,418.76028,5809470.36103,1721426.00458,4536.51638,2474.06794,2062.89925,163508.92065,386.65805,40.75438,-73.95706,2178124473.71928,0.01841
std,4308.45665,1577.52941,30.53021,29.98008,8623.94060,9803.33595,127.97207,10550.54438,30967891.53918,9219300.81457,204590.94970,195861.05586,59429.02405,291282.07463,9330.86855,0.08012,0.04634,1222326211.19575,0.47081
min,1.00000,1.00000,1600.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-23134.30000,0.00000,0.00000,40.51607,-74.24358,1000047501.00000,0.00000
25%,3464.50000,150.00000,1927.00000,37.00000,61.80000,65.10000,3.80000,103.50000,1043434.65000,301634.90000,328.50000,147.70000,94.70000,66926.50000,99.40000,40.70723,-73.98466,1012930048.00000,0.00413
50%,7020.00000,446.00000,1941.00000,65.00000,78.50000,82.40000,5.30000,129.30000,1848203.90000,539821.20000,498.70000,272.25000,171.30000,93725.00000,124.80000,40.75913,-73.96281,2028770166.00000,0.00528
75%,11065.50000,1304.50000,1966.00000,85.00000,97.50000,102.40000,9.20000,167.00000,4339162.35000,1274200.10000,902.00000,446.57500,422.10000,157481.00000,162.50000,40.81762,-73.93244,3064148753.50000,0.00680
max,14993.00000,76179.00000,2019.00000,100.00000,869265.00000,939329.00000,6259.40000,986366.00000,1691763071.00000,495827334.50000,20943400.00000,20943400.00000,4764375.00000,14217119.00000,912801.10000,40.91287,-73.71554,5079910100.00000,46.17222


In [272]:
# rename some of the columns to compare datasets across years more easily
filtered2016.rename(columns={'Order': 'order',
                             'Property Name': 'prop_name',
                             'Address 1 (self-reported)': 'address',
                             'Street Number': 'strt_num',
                             'Street Name': 'strt_name',
                             'Borough': 'borough',
                             'Property GFA - Self-Reported (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI (kBtu/ft²)': 'src_eui',
                             'Weather Normalized Source EUI (kBtu/ft²)': 'wnsrc_eui',
                             'Site EUI (kBtu/ft²)': 'site_eui',
                             'Weather Normalized Site EUI (kBtu/ft²)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions (Metric Tons CO2e)': 'total_ghg',
                             'Direct GHG Emissions (Metric Tons CO2e)': 'direct_ghg',
                             'Indirect GHG Emissions (Metric Tons CO2e)': 'indrct_ghg',
                             'Latitude': 'latitude',
                             'Longitude': 'longitude',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)
filtered2016.head()

,order,prop_name,address,strt_num,strt_name,borough,prop_type,year_built,energ_star,site_eui,wnsite_eui,wn_sei_kwh,wnsrc_eui,eu_gp_kbtu,wnse_kwh,total_ghg,direct_ghg,indrct_ghg,sq_footage,src_eui,latitude,longitude,bbl,t_ghg_psqf
0,1,201/205,201/205 East 42nd st.,675.00000,3 AVENUE,Manhattan,Office,1963,nan,305.60000,303.10000,37.80000,614.20000,38139374.20000,11082770.50000,6962.20000,0.00000,6962.20000,762051,619.40000,40.75079,-73.97396,1013160001,0.00914
1,2,NYP Columbia (West Campus),622 168th Street,180.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1969,55.00000,229.80000,228.80000,24.80000,401.10000,332365924.00000,96261312.10000,55870.40000,51016.40000,4854.10000,3889181,404.30000,40.84140,-73.94257,1021380040,0.01437
2,3,MSCHoNY North,3975 Broadway,3975.00000,BROADWAY,Manhattan,Hospital (General Medical & Surgical),1924,nan,nan,nan,nan,nan,nan,nan,0.00000,0.00000,0.00000,231342,nan,40.84043,-73.94025,1021380030,0.00000
3,4,Herbert Irving Pavilion & Millstein Hospital,161 Fort Washington Ave,161.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1971,nan,nan,nan,nan,nan,nan,nan,0.00000,0.00000,0.00000,1305748,nan,40.84075,-73.94285,1021390001,0.00000
4,5,Neuro Institute,710 West 168th Street,193.00000,FT WASHINGTON AVENUE,Manhattan,Hospital (General Medical & Surgical),1932,nan,nan,nan,nan,nan,nan,nan,0.00000,0.00000,0.00000,179694,nan,40.84156,-73.94253,1021390085,0.00000


In [273]:
filtered2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11695 entries, 0 to 11745
Data columns (total 24 columns):
order         11695 non-null int64
prop_name     11695 non-null object
address       11695 non-null object
strt_num      9560 non-null float64
strt_name     11624 non-null object
borough       11628 non-null object
prop_type     11695 non-null object
year_built    11695 non-null int64
energ_star    9618 non-null float64
site_eui      11538 non-null float64
wnsite_eui    10242 non-null float64
wn_sei_kwh    10918 non-null float64
wnsrc_eui     10242 non-null float64
eu_gp_kbtu    11459 non-null float64
wnse_kwh      10918 non-null float64
total_ghg     11623 non-null float64
direct_ghg    11614 non-null float64
indrct_ghg    11633 non-null float64
sq_footage    11695 non-null int64
src_eui       11538 non-null float64
latitude      9483 non-null float64
longitude     9483 non-null float64
bbl           11695 non-null int64
t_ghg_psqf    11623 non-null float64
dtypes: float64(15),

In [274]:
data2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13223 entries, 0 to 13222
Data columns (total 64 columns):
Record Number                                                      11531 non-null float64
Order                                                              13223 non-null int64
NYC Borough, Block and Lot (BBL)                                   13222 non-null float64
Co-reported BBL Status                                             615 non-null object
BBLs Co-reported                                                   615 non-null object
Reported NYC Building Identification Numbers (BINs)                11978 non-null object
Property Name                                                      11531 non-null object
Parent Property Id                                                 11531 non-null object
Parent Property Name                                               11531 non-null object
Street Number                                                      10380 non-null float64
Str

In [275]:
# In 2015, our BBL data is already a non-string object so we don't need to do all the transformations
# we did for 2017 and 2018
data2015['BBL'] = data2015['NYC Borough, Block and Lot (BBL)']
filtered2015 = data2015[data2015['BBL'].notnull()]
filtered2015.head()

,Record Number,Order,"NYC Borough, Block and Lot (BBL)",Co-reported BBL Status,BBLs Co-reported,Reported NYC Building Identification Numbers (BINs),Property Name,Parent Property Id,Parent Property Name,Street Number,Street Name,Postcode,Borough,DOF Benchmarking Submission Status,Primary Property Type - Self Selected,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),DOF Property Floor Area (ft²),Property GFA - Self-reported (ft²),Water Use (All Water Sources) (kgal),Municipally Supplied Potable Water - Indoor Intensity (gal/ft²),Release Date,DEP Provided Water Use (kgal),Automatic Water Benchmarking Eligible,Reported Water Method,Latitude,Longitude,Community Board,Council District,Census Tract,NTA,BBL
0,4113711.00000,1,1003360005.00000,NaN,NaN,1088884,16 Bialystoker Place,Not Applicable: Standalone Property,Not Applicable: Standalone Property,15.00000,BIALYSTOKER PLACE,10002.00000,Manhattan,In Compliance,Multifamily Housing,Multifamily Housing,Multifamily Housing,98000.00000,Not Available,Not Available,Not Available,Not Available,1982.00000,1.00000,100.00000,Whole Building,Not Available,87.00000,73.70000,74.40000,5.30000,0.60000,115.00000,115.70000,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,5463399.10000,55261.40000,1762214.10000,516475.30000,436.30000,290.20000,146.10000,98052.00000,98000.00000,nan,nan,04/13/2016 11:08:48 AM,nan,NaN,NaN,40.71566,-73.98298,3.00000,1.00000,12.00000,Lower East Side ...,1003360005.00000
1,4630135.00000,2,1010617501.00000,NaN,NaN,1026728,(7478) - 432 West 52nd Street Condominium,Not Applicable: Standalone Property,Not Applicable: Standalone Property,432.00000,WEST 52 STREET,10019.00000,Manhattan,In Compliance,Multifamily Housing,Multifamily Housing,Multifamily Housing,57401.00000,Not Available,Not Available,Not Available,Not Available,1950.00000,1.00000,95.00000,Whole Building,Whole Building,79.00000,51.30000,50.90000,7.50000,0.30000,108.80000,107.10000,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,1437732.80000,14515.60000,1507302.80000,431480.20000,201.40000,76.40000,125.00000,58851.00000,57401.00000,768.50000,13.39000,04/26/2016 09:52:05 PM,nan,Not Eligible,NaN,40.76518,-73.98912,4.00000,3.00000,133.00000,Clinton ...,1010617501.00000
2,4626594.00000,3,1014270028.00000,NaN,NaN,1043974,(9250) - 250 East 73rd Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,1393.00000,2 AVENUE,10021.00000,Manhattan,In Compliance,Multifamily Housing,Multifamily Housing,Multifamily Housing,166433.00000,Not Available,Not Available,Not Available,Not Available,1964.00000,1.00000,95.00000,Whole Building,Whole Building,80.00000,44.90000,44.50000,8.20000,0.20000,106.70000,105.20000,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,2739020.00000,273

In [276]:
# drop columns we won't visualize to reduce file size
filtered2015.drop(['Record Number','NYC Borough, Block and Lot (BBL)', 'Co-reported BBL Status', 'BBLs Co-reported', 'Reported NYC Building Identification Numbers (BINs)', 'Parent Property Id', 'Parent Property Name', 'Postcode', 'List of All Property Use Types at Property', 'Largest Property Use Type', 'Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', '2nd Largest Property Use - Gross Floor Area (ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Occupancy', 'Metered Areas (Energy)', 'Metered Areas  (Water)', 'Fuel Oil #1 Use (kBtu)', 'Fuel Oil #2 Use (kBtu)', 'Fuel Oil #4 Use (kBtu)', 'Fuel Oil #5 & 6 Use (kBtu)', 'Diesel #2 Use (kBtu)', 'Natural Gas Use (kBtu)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Site Natural Gas Use (therms)', 'District Steam Use (kBtu)', 'Water Use (All Water Sources) (kgal)', 'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)', 'District Hot Water Use (kBtu)', 'District Chilled Water Use (kBtu)',  'Release Date', 'DEP Provided Water Use (kgal)', 'Automatic Water Benchmarking Eligible', 'Reported Water Method', 'Property GFA - Self-reported (ft²)', 'DOF Benchmarking Submission Status', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'NTA', 'Release Date', 'Number of Buildings - Self-reported'], axis = 1, inplace = True)
filtered2015.head()


,Order,Property Name,Street Number,Street Name,Borough,Primary Property Type - Self Selected,Year Built,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),DOF Property Floor Area (ft²),BBL
0,1,16 Bialystoker Place,15.00000,BIALYSTOKER PLACE,Manhattan,Multifamily Housing,1982.00000,87.00000,73.70000,74.40000,5.30000,115.00000,115.70000,1762214.10000,516475.30000,436.30000,290.20000,146.10000,98052.00000,1003360005.00000
1,2,(7478) - 432 West 52nd Street Condominium,432.00000,WEST 52 STREET,Manhattan,Multifamily Housing,1950.00000,79.00000,51.30000,50.90000,7.50000,108.80000,107.10000,1507302.80000,431480.20000,201.40000,76.40000,125.00000,58851.00000,1010617501.00000
2,3,(9250) - 250 East 73rd Street,1393.00000,2 AVENUE,Manhattan,Multifamily Housing,1964.00000,80.00000,44.90000,44.50000,8.20000,106.70000,105.20000,4737201.30000,1365852.00000,538.40000,145.50000,392.90000,166432.00000,1014270028.00000
3,4,(9141) - 141 East 89th Street,1341.00000,LEXINGTON AVENUE,Manhattan,Multifamily Housing,1960.00000,10.00000,91.30000,90.80000,12.00000,182.40000,180.70000,4759700.80000,1377116.10000,699.30000,304.60000,394.70000,114939.00000,1015180024.00000
4,5,605 East 9th ST,350.00000,EAST 10 STREET,Manhattan,Residence Hall/Dormitory,2018.00000,nan,nan,nan,nan,nan,nan,nan,nan,0.00000,0.00000,0.00000,93420.00000,1003920010.00000


In [277]:
# pd.to_numeric(arg, errors=’raise’, downcast=None)
# convert columns to floats or ints if that's what they contain
# filtered2015['Year Built'].value_counts()
cols_to_number = ['DOF Property Floor Area (ft²)', 'Year Built', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Electricity Use - Grid Purchase (kBtu)', 'Weather Normalized Site Electricity (kWh)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Indirect GHG Emissions (Metric Tons CO2e)' ]
filtered2015[cols_to_number] = filtered2015[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2015['BBL'] = filtered2015['BBL'].astype(int)
filtered2015.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Order,Street Number,Year Built,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),DOF Property Floor Area (ft²),BBL
count,13222.00000,10380.00000,11530.00000,9534.00000,11438.00000,9848.00000,11095.00000,11438.00000,9848.00000,11424.00000,11097.00000,11477.00000,11470.00000,11494.00000,12627.00000,13222.00000
mean,6612.14483,996.28825,1949.13903,57.73914,525.76930,527.60576,19.21406,733.20571,735.28634,8200947.98676,2380524.73542,6952.72290,1340.27280,5605.56771,163677.29540,2198121493.99675
std,3817.40367,1550.76245,30.26219,30.14353,10120.54684,9761.99062,384.60039,11991.08719,11466.35518,121469611.77452,35765463.09121,169230.47748,59493.33027,158331.48963,299351.87949,1218340038.21976
min,1.00000,0.00000,1600.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-23358.60000,50008.00000,1000047501.00000
25%,3306.25000,146.00000,1927.00000,34.00000,65.30000,65.70000,3.80000,103.20000,103.80000,1045688.05000,302619.00000,342.00000,171.50000,86.30000,65000.00000,1013110029.25000
50%,6612.50000,441.00000,1941.00000,63.00000,82.40000,82.50000,5.30000,129.20000,129.20000,1885737.00000,545467.40000,519.80000,305.05000,158.00000,92400.00000,2029920047.50000
75%,9917.75000,1300.25000,1967.00000,83.00000,103.00000,102.32500,8.90000,168.57500,167.92500,4513345.92500,1311118.80000,939.30000,504.95000,406.67500,156355.50000,3061217501.50000
max,13223.00000,76179.00000,2019.00000,100.00000,801504.70000,798135.70000,24369.50000,841652.40000,838112.30000,10476204690.00000,3046183035.50000,15014684.70000,5852949.00000,15014550.00000,13435076.00000,5080080134.00000


In [278]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2015['Emissions Per Sq Ft'] = np.where(filtered2015['DOF Property Floor Area (ft²)'] <= 0, 0, filtered2015['Total GHG Emissions (Metric Tons CO2e)'] /filtered2015['DOF Property Floor Area (ft²)'])
filtered2015.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Order,Street Number,Year Built,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),DOF Property Floor Area (ft²),BBL,Emissions Per Sq Ft
count,13222.00000,10380.00000,11530.00000,9534.00000,11438.00000,9848.00000,11095.00000,11438.00000,9848.00000,11424.00000,11097.00000,11477.00000,11470.00000,11494.00000,12627.00000,13222.00000,11258.00000
mean,6612.14483,996.28825,1949.13903,57.73914,525.76930,527.60576,19.21406,733.20571,735.28634,8200947.98676,2380524.73542,6952.72290,1340.27280,5605.56771,163677.29540,2198121493.99675,0.03127
std,3817.40367,1550.76245,30.26219,30.14353,10120.54684,9761.99062,384.60039,11991.08719,11466.35518,121469611.77452,35765463.09121,169230.47748,59493.33027,158331.48963,299351.87949,1218340038.21976,0.57138
min,1.00000,0.00000,1600.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-23358.60000,50008.00000,1000047501.00000,0.00000
25%,3306.25000,146.00000,1927.00000,34.00000,65.30000,65.70000,3.80000,103.20000,103.80000,1045688.05000,302619.00000,342.00000,171.50000,86.30000,65000.00000,1013110029.25000,0.00431
50%,6612.50000,441.00000,1941.00000,63.00000,82.40000,82.50000,5.30000,129.20000,129.20000,1885737.00000,545467.40000,519.80000,305.05000,158.00000,92400.00000,2029920047.50000,0.00545
75%,9917.75000,1300.25000,1967.00000,83.00000,103.00000,102.32500,8.90000,168.57500,167.92500,4513345.92500,1311118.80000,939.30000,504.95000,406.67500,156355.50000,3061217501.50000,0.00700
max,13223.00000,76179.00000,2019.00000,100.00000,801504.70000,798135.70000,24369.50000,841652.40000,838112.30000,10476204690.00000,3046183035.50000,15014684.70000,5852949.00000,15014550.00000,13435076.00000,5080080134.00000,39.19031


In [279]:
# rename some of the columns to compare datasets across years more easily
filtered2015.rename(columns={'Order': 'order',
                             'Property Name': 'prop_name',
#                              'Address 1 (self-reported)': 'address',
                             'Street Number': 'strt_num',
                             'Street Name': 'strt_name',
                             'Borough': 'borough',
                             'DOF Property Floor Area (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI (kBtu/ft²)': 'src_eui',
                             'Weather Normalized Source EUI (kBtu/ft²)': 'wnsrc_eui',
                             'Site EUI (kBtu/ft²)': 'site_eui',
                             'Weather Normalized Site EUI (kBtu/ft²)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions (Metric Tons CO2e)': 'total_ghg',
                             'Direct GHG Emissions (Metric Tons CO2e)': 'direct_ghg',
                             'Indirect GHG Emissions (Metric Tons CO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)
filtered2015.head()

,order,prop_name,strt_num,strt_name,borough,prop_type,year_built,energ_star,site_eui,wnsite_eui,wn_sei_kwh,src_eui,wnsrc_eui,eu_gp_kbtu,wnse_kwh,total_ghg,direct_ghg,indrct_ghg,sq_footage,bbl,t_ghg_psqf
0,1,16 Bialystoker Place,15.00000,BIALYSTOKER PLACE,Manhattan,Multifamily Housing,1982.00000,87.00000,73.70000,74.40000,5.30000,115.00000,115.70000,1762214.10000,516475.30000,436.30000,290.20000,146.10000,98052.00000,1003360005,0.00445
1,2,(7478) - 432 West 52nd Street Condominium,432.00000,WEST 52 STREET,Manhattan,Multifamily Housing,1950.00000,79.00000,51.30000,50.90000,7.50000,108.80000,107.10000,1507302.80000,431480.20000,201.40000,76.40000,125.00000,58851.00000,1010617501,0.00342
2,3,(9250) - 250 East 73rd Street,1393.00000,2 AVENUE,Manhattan,Multifamily Housing,1964.00000,80.00000,44.90000,44.50000,8.20000,106.70000,105.20000,4737201.30000,1365852.00000,538.40000,145.50000,392.90000,166432.00000,1014270028,0.00323
3,4,(9141) - 141 East 89th Street,1341.00000,LEXINGTON AVENUE,Manhattan,Multifamily Housing,1960.00000,10.00000,91.30000,90.80000,12.00000,182.40000,180.70000,4759700.80000,1377116.10000,699.30000,304.60000,394.70000,114939.00000,1015180024,0.00608
4,5,605 East 9th ST,350.00000,EAST 10 STREET,Manhattan,Residence Hall/Dormitory,2018.00000,nan,nan,nan,nan,nan,nan,nan,nan,0.00000,0.00000,0.00000,93420.00000,1003920010,0.00000


In [280]:
filtered2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13222 entries, 0 to 13222
Data columns (total 21 columns):
order         13222 non-null int64
prop_name     11530 non-null object
strt_num      10380 non-null float64
strt_name     13222 non-null object
borough       13222 non-null object
prop_type     11977 non-null object
year_built    11530 non-null float64
energ_star    9534 non-null float64
site_eui      11438 non-null float64
wnsite_eui    9848 non-null float64
wn_sei_kwh    11095 non-null float64
src_eui       11438 non-null float64
wnsrc_eui     9848 non-null float64
eu_gp_kbtu    11424 non-null float64
wnse_kwh      11097 non-null float64
total_ghg     11477 non-null float64
direct_ghg    11470 non-null float64
indrct_ghg    11494 non-null float64
sq_footage    12627 non-null float64
bbl           13222 non-null int64
t_ghg_psqf    11258 non-null float64
dtypes: float64(15), int64(2), object(4)
memory usage: 2.2+ MB


In [281]:
data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 33 columns):
Record Number                                                      11804 non-null float64
BBL                                                                13974 non-null int64
Co-reported BBL Status                                             668 non-null object
BBLs Co-reported                                                   668 non-null object
Reported NYC Building Identification Numbers (BINs)                11504 non-null object
Street Number                                                      13634 non-null object
Street Name                                                        13678 non-null object
Borough                                                            13974 non-null object
Postcode                                                           11614 non-null float64
BBL on the Covered Buildings List                                  13974 non-null object
DOF 

In [282]:
# In 2014, our BBL data is already a non-string object so we don't need to do all the transformations
# we did for 2017 and 2018
filtered2014 = data2014[data2014['BBL'].notnull()]
filtered2014.head()

,Record Number,BBL,Co-reported BBL Status,BBLs Co-reported,Reported NYC Building Identification Numbers (BINs),Street Number,Street Name,Borough,Postcode,BBL on the Covered Buildings List,DOF Benchmarking Submission Status,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),Municipally Supplied Potable Water - Indoor Intensity (gal/ft²),Automatic Water Benchmarking Eligible,Reported Water Method,ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),DOF Property Floor Area (Buildngs and Parking)(ft2),Primary Property Type - Self Selected,DOF Number of Buildings,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,2830985.00000,4160830037,NaN,NaN,4303321,71,BEACH 70 STREET,QUEENS,11692.00000,Yes,In Compliance,115.10000,Not Available,146,Not Available,Not Available,No,None,20,2362.9,2026.1,336.8,312100,155000.00000,Multifamily Housing,1.00000,nan,nan,nan,nan,nan,nan,NaN
1,2858535.00000,4002520018,NaN,NaN,4003447,32-36,47 AVENUE,QUEENS,11101.00000,Yes,In Compliance,14.80000,12,18.6,15.5,0.05,Yes,ABS,100,434.6,379.8,54.8,390400,184400.00000,Office,1.00000,nan,nan,nan,nan,nan,nan,NaN
2,2804445.00000,3009747504,NaN,NaN,3392416,302,2 AVENUE,BROOKLYN,11215.00000,Yes,In Compliance,102.30000,101.9,188.8,187.6,52.15,No,Manual,9,387,197.4,189.6,58676,58506.00000,Multifamily Housing,1.00000,nan,nan,nan,nan,nan,nan,NaN
3,2805372.00000,4032937501,NaN,NaN,4308252,110,73 ROAD,QUEENS,11375.00000,Yes,In Compliance,44.60000,42.9,63.9,62,39.6,Yes,ABS,Not Available,289.9,214.9,75,110880,100800.00000,Multifamily Housing,1.00000,nan,nan,nan,nan,nan,nan,NaN
4,2664866.00000,4000540006,NaN,NaN,4000460,11-36,45 ROAD,QUEENS,11101.00000,Yes,In Compliance,25.80000,24.7,64,62.8,6.03,Yes,ABS,95,107.7,24.8,83,56777,56000.00000,Worship Facility,1.00000,nan,nan,nan,nan,nan,nan,NaN


In [283]:
# drop columns we won't visualize to reduce file size
filtered2014.drop(['Record Number', 'Co-reported BBL Status', 'BBLs Co-reported', 'Reported NYC Building Identification Numbers (BINs)', 'Postcode', 'BBL on the Covered Buildings List', 'DOF Benchmarking Submission Status', 'Automatic Water Benchmarking Eligible','Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)', 'Reported Water Method', 'DOF Number of Buildings', 'DOF Property Floor Area (Buildngs and Parking)(ft2)', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'NTA'], axis = 1, inplace = True)
filtered2014.head()

,BBL,Street Number,Street Name,Borough,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),Primary Property Type - Self Selected
0,4160830037,71,BEACH 70 STREET,QUEENS,115.10000,Not Available,146,Not Available,20,2362.9,2026.1,336.8,312100,Multifamily Housing
1,4002520018,32-36,47 AVENUE,QUEENS,14.80000,12,18.6,15.5,100,434.6,379.8,54.8,390400,Office
2,3009747504,302,2 AVENUE,BROOKLYN,102.30000,101.9,188.8,187.6,9,387,197.4,189.6,58676,Multifamily Housing
3,4032937501,110,73 ROAD,QUEENS,44.60000,42.9,63.9,62,Not Available,289.9,214.9,75,110880,Multifamily Housing
4,4000540006,11-36,45 ROAD,QUEENS,25.80000,24.7,64,62.8,95,107.7,24.8,83,56777,Worship Facility


In [284]:
# pd.to_numeric(arg, errors=’raise’, downcast=None)
# convert columns to floats or ints if that's what they contain
# filtered2015['Year Built'].value_counts()
cols_to_number = ['Reported Property Floor Area (Building(s)) (ft²)', 'ENERGY STAR Score', 'Source EUI(kBtu/ft2)', 'Weather Normalized Source EUI(kBtu/ft2)', 'Site EUI(kBtu/ft2)', 'Weather Normalized Site EUI(kBtu/ft2)', 'Total GHG Emissions(MtCO2e)', 'Direct GHG Emissions(MtCO2e)', 'Indirect GHG Emissions(MtCO2e)' ]
filtered2014[cols_to_number] = filtered2014[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2014['BBL'] = filtered2014['BBL'].astype(int)
filtered2014.describe()


,BBL,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²)
count,13974.00000,10610.00000,8712.00000,10610.00000,8712.00000,8506.00000,10689.00000,10799.00000,10992.00000,11302.00000
mean,2211436437.78803,2412.01103,2764.84329,2968.26358,3388.10293,55.34693,16682.59572,759.06807,15484.63944,176671.50257
std,1225263923.14149,142814.51796,157371.29176,171814.27748,189323.63374,30.15663,774936.37795,16929.25830,764013.56134,316363.13722
min,1000010010.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,1013100060.00000,68.00000,65.20000,105.90000,103.60000,31.00000,361.10000,181.15000,83.70000,67620.75000
50%,2030840023.00000,86.10000,82.50000,132.30000,129.30000,59.00000,553.40000,329.10000,153.95000,96157.00000
75%,3061322501.00000,106.90000,102.50000,171.20000,168.90000,81.00000,1026.10000,544.80000,407.57500,165462.50000
max,5080080014.00000,10804120.00000,10804120.00000,12997870.00000,12997870.00000,100.00000,60521600.00000,1745981.00000,60521600.00000,14217119.00000


In [285]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2014['Emissions Per Sq Ft'] = np.where(filtered2014['Reported Property Floor Area (Building(s)) (ft²)'] <= 0, 0, filtered2014['Total GHG Emissions(MtCO2e)'] /filtered2014['Reported Property Floor Area (Building(s)) (ft²)'])
filtered2014.describe()


,BBL,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),Emissions Per Sq Ft
count,13974.00000,10610.00000,8712.00000,10610.00000,8712.00000,8506.00000,10689.00000,10799.00000,10992.00000,11302.00000,10677.00000
mean,2211436437.78803,2412.01103,2764.84329,2968.26358,3388.10293,55.34693,16682.59572,759.06807,15484.63944,176671.50257,0.15972
std,1225263923.14149,142814.51796,157371.29176,171814.27748,189323.63374,30.15663,774936.37795,16929.25830,764013.56134,316363.13722,9.45239
min,1000010010.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,1013100060.00000,68.00000,65.20000,105.90000,103.60000,31.00000,361.10000,181.15000,83.70000,67620.75000,0.00442
50%,2030840023.00000,86.10000,82.50000,132.30000,129.30000,59.00000,553.40000,329.10000,153.95000,96157.00000,0.00568
75%,3061322501.00000,106.90000,102.50000,171.20000,168.90000,81.00000,1026.10000,544.80000,407.57500,165462.50000,0.00730
max,5080080014.00000,10804120.00000,10804120.00000,12997870.00000,12997870.00000,100.00000,60521600.00000,1745981.00000,60521600.00000,14217119.00000,717.37806


In [287]:
# rename some of the columns to compare datasets across years more easily
filtered2014.rename(columns={'Order': 'order',
#                              'Property Name': 'prop_name',
#                              'Address 1 (self-reported)': 'address',
                             'Street Number': 'strt_num',
                             'Street Name': 'strt_name',
                             'Borough': 'borough',
                             'Reported Property Floor Area (Building(s)) (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI(kBtu/ft2)': 'src_eui',
                             'Weather Normalized Source EUI(kBtu/ft2)': 'wnsrc_eui',
                             'Site EUI(kBtu/ft2)': 'site_eui',
                             'Weather Normalized Site EUI(kBtu/ft2)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions(MtCO2e)': 'total_ghg',
                             'Direct GHG Emissions(MtCO2e)': 'direct_ghg',
                             'Indirect GHG Emissions(MtCO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)

filtered2014.head()


,bbl,strt_num,strt_name,borough,site_eui,wnsite_eui,src_eui,wnsrc_eui,energ_star,total_ghg,direct_ghg,indrct_ghg,sq_footage,prop_type,t_ghg_psqf
0,4160830037,71,BEACH 70 STREET,QUEENS,115.10000,nan,146.00000,nan,20.00000,2362.90000,2026.10000,336.80000,312100.00000,Multifamily Housing,0.00757
1,4002520018,32-36,47 AVENUE,QUEENS,14.80000,12.00000,18.60000,15.50000,100.00000,434.60000,379.80000,54.80000,390400.00000,Office,0.00111
2,3009747504,302,2 AVENUE,BROOKLYN,102.30000,101.90000,188.80000,187.60000,9.00000,387.00000,197.40000,189.60000,58676.00000,Multifamily Housing,0.00660
3,4032937501,110,73 ROAD,QUEENS,44.60000,42.90000,63.90000,62.00000,nan,289.90000,214.90000,75.00000,110880.00000,Multifamily Housing,0.00261
4,4000540006,11-36,45 ROAD,QUEENS,25.80000,24.70000,64.00000,62.80000,95.00000,107.70000,24.80000,83.00000,56777.00000,Worship Facility,0.00190


In [288]:
filtered2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13974 entries, 0 to 13973
Data columns (total 15 columns):
bbl           13974 non-null int64
strt_num      13634 non-null object
strt_name     13678 non-null object
borough       13974 non-null object
site_eui      10610 non-null float64
wnsite_eui    8712 non-null float64
src_eui       10610 non-null float64
wnsrc_eui     8712 non-null float64
energ_star    8506 non-null float64
total_ghg     10689 non-null float64
direct_ghg    10799 non-null float64
indrct_ghg    10992 non-null float64
sq_footage    11302 non-null float64
prop_type     13678 non-null object
t_ghg_psqf    10677 non-null float64
dtypes: float64(10), int64(1), object(4)
memory usage: 1.7+ MB


In [289]:
data2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16170 entries, 0 to 16169
Data columns (total 32 columns):
Record Number                                                      16170 non-null int64
BBL                                                                16014 non-null float64
Co-reported BBL Status                                             912 non-null object
BBLs Co-reported                                                   13790 non-null object
Reported NYC Building Identificaiton Numbers (BINs)                13790 non-null object
Street Number                                                      14423 non-null object
Street Name                                                        14458 non-null object
Borough                                                            16170 non-null object
Postcode                                                           14432 non-null float64
DOF Benchmarking Submission Status                                 16170 non-null object
Si

In [290]:
# In 2013, our BBL data is already a non-string object so we don't need to do all the transformations
# we did for 2017 and 2018
filtered2013 = data2013[data2013['BBL'].notnull()]
filtered2013.head()

,Record Number,BBL,Co-reported BBL Status,BBLs Co-reported,Reported NYC Building Identificaiton Numbers (BINs),Street Number,Street Name,Borough,Postcode,DOF Benchmarking Submission Status,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),Municipally Supplied Potable Water - Indoor Intensity (gal/ft²),Automatic Water Benchmarking Eligible,Reported Water Method,ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),DOF Property Floor Area (Buildngs and Parking)(ft2),Primary Property Type - Self Selected,DOF Number of Buildings,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,12086,1000410001.00000,NaN,1000410001,1001007,NaN,NaN,Manhattan,nan,BBL not valid,NaN,Not Available,NaN,Not Available,Not Available,NaN,NaN,Not Available,NaN,NaN,NaN,918919,nan,Office,nan,nan,nan,nan,nan,nan,nan,NaN
1,14354,4021460028.00000,NaN,NaN,NaN,63-10,108 STREET,Queens,11375.00000,No Submission Matched to BBL,NaN,NaN,NaN,NaN,NaN,Yes,None,NaN,NaN,NaN,NaN,NaN,82730.00000,NaN,1.00000,nan,nan,nan,nan,nan,nan,NaN
2,12150,2027620097.00000,NaN,2027620097,2006460,NaN,NaN,Bronx,nan,BBL not valid,NaN,Not Available,NaN,Not Available,Not Available,NaN,NaN,Not Available,NaN,NaN,NaN,64200,nan,Multifamily Housing,nan,nan,nan,nan,nan,nan,nan,NaN
3,12590,1006700001.00000,NaN,NaN,NaN,201,11 AVENUE,Manhattan,10001.00000,No Submission Matched to BBL,NaN,NaN,NaN,NaN,NaN,No,None,NaN,NaN,NaN,NaN,NaN,440360.00000,NaN,1.00000,40.74965,-74.00647,4.00000,3.00000,99.00000,1012266.00000,Hudson Yards-Chelsea-Flatiron-Union Square ...
4,15591,2048300034.00000,NaN,2048300034,2063117,NaN,NaN,Bronx,nan,Not on Covered Buildings List,NaN,Not Available,NaN,Not Available,Not Available,NaN,None,Not Available,NaN,NaN,NaN,50000,nan,Multifamily Housing,nan,nan,nan,nan,nan,nan,nan,NaN


In [291]:
# drop columns we won't visualize to reduce file size
filtered2013.drop(['Record Number', 'Co-reported BBL Status', 'BBLs Co-reported', 'Reported NYC Building Identificaiton Numbers (BINs)', 'Postcode', 'DOF Benchmarking Submission Status', 'Automatic Water Benchmarking Eligible','Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)', 'Reported Water Method', 'DOF Number of Buildings', 'DOF Property Floor Area (Buildngs and Parking)(ft2)', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'NTA'], axis = 1, inplace = True)
filtered2013.head()

,BBL,Street Number,Street Name,Borough,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),Primary Property Type - Self Selected
0,1000410001.00000,NaN,NaN,Manhattan,NaN,Not Available,NaN,Not Available,Not Available,NaN,NaN,NaN,918919,Office
1,4021460028.00000,63-10,108 STREET,Queens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2027620097.00000,NaN,NaN,Bronx,NaN,Not Available,NaN,Not Available,Not Available,NaN,NaN,NaN,64200,Multifamily Housing
3,1006700001.00000,201,11 AVENUE,Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2048300034.00000,NaN,NaN,Bronx,NaN,Not Available,NaN,Not Available,Not Available,NaN,NaN,NaN,50000,Multifamily Housing


In [292]:
# pd.to_numeric(arg, errors=’raise’, downcast=None)
# convert columns to floats or ints if that's what they contain
# filtered2015['Year Built'].value_counts()
cols_to_number = ['Reported Property Floor Area (Building(s)) (ft²)', 'ENERGY STAR Score', 'Source EUI(kBtu/ft2)', 'Weather Normalized Source EUI(kBtu/ft2)', 'Site EUI(kBtu/ft2)', 'Weather Normalized Site EUI(kBtu/ft2)', 'Total GHG Emissions(MtCO2e)', 'Direct GHG Emissions(MtCO2e)', 'Indirect GHG Emissions(MtCO2e)' ]
filtered2013[cols_to_number] = filtered2013[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2013['BBL'] = filtered2013['BBL'].astype(int)
filtered2013.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,BBL,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²)
count,16014.00000,11467.00000,9745.00000,11467.00000,9745.00000,2002.00000,11603.00000,11926.00000,12103.00000,12906.00000
mean,2240853668.44361,3056.50713,3551.88026,3730.38235,4323.14494,60.80420,67039.41243,516.57234,63796.65155,174336.80048
std,1234668834.26278,265112.77901,287580.66774,318942.37102,345971.52871,29.26939,6267993.71481,1709.67320,6137164.40051,312867.70156
min,1000010010.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,-34260.40000,0.00000
25%,1013320008.00000,65.20000,64.60000,103.95000,103.60000,41.00000,332.55000,127.42500,79.90000,65627.75000
50%,2031565088.00000,84.10000,83.10000,131.40000,130.10000,68.00000,521.10000,299.35000,150.90000,93371.50000
75%,3065167522.00000,106.10000,104.20000,171.90000,170.70000,84.00000,968.30000,506.57500,396.10000,163397.75000
max,5079910100.00000,28226010.00000,28226010.00000,33957080.00000,33957080.00000,100.00000,674366502.40000,88264.60000,674366500.00000,14217119.00000


In [293]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2013['Emissions Per Sq Ft'] = np.where(filtered2013['Reported Property Floor Area (Building(s)) (ft²)'] <= 0, 0, filtered2013['Total GHG Emissions(MtCO2e)'] /filtered2013['Reported Property Floor Area (Building(s)) (ft²)'])
filtered2013.describe()

/Users/genevieve/anaconda3/envs/opendataweek/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,BBL,Site EUI(kBtu/ft2),Weather Normalized Site EUI(kBtu/ft2),Source EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Direct GHG Emissions(MtCO2e),Indirect GHG Emissions(MtCO2e),Reported Property Floor Area (Building(s)) (ft²),Emissions Per Sq Ft
count,16014.00000,11467.00000,9745.00000,11467.00000,9745.00000,2002.00000,11603.00000,11926.00000,12103.00000,12906.00000,11546.00000
mean,2240853668.44361,3056.50713,3551.88026,3730.38235,4323.14494,60.80420,67039.41243,516.57234,63796.65155,174336.80048,0.26682
std,1234668834.26278,265112.77901,287580.66774,318942.37102,345971.52871,29.26939,6267993.71481,1709.67320,6137164.40051,312867.70156,23.39227
min,1000010010.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,-34260.40000,0.00000,0.00000
25%,1013320008.00000,65.20000,64.60000,103.95000,103.60000,41.00000,332.55000,127.42500,79.90000,65627.75000,0.00424
50%,2031565088.00000,84.10000,83.10000,131.40000,130.10000,68.00000,521.10000,299.35000,150.90000,93371.50000,0.00564
75%,3065167522.00000,106.10000,104.20000,171.90000,170.70000,84.00000,968.30000,506.57500,396.10000,163397.75000,0.00740
max,5079910100.00000,28226010.00000,28226010.00000,33957080.00000,33957080.00000,100.00000,674366502.40000,88264.60000,674366500.00000,14217119.00000,2499.08837


In [294]:
# rename some of the columns to compare datasets across years more easily
filtered2013.rename(columns={'Order': 'order',
#                              'Property Name': 'prop_name',
#                              'Address 1 (self-reported)': 'address',
                             'Street Number': 'strt_num',
                             'Street Name': 'strt_name',
                             'Borough': 'borough',
                             'Reported Property Floor Area (Building(s)) (ft²)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
                             'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
                             'Source EUI(kBtu/ft2)': 'src_eui',
                             'Weather Normalized Source EUI(kBtu/ft2)': 'wnsrc_eui',
                             'Site EUI(kBtu/ft2)': 'site_eui',
                             'Weather Normalized Site EUI(kBtu/ft2)': 'wnsite_eui',
                             'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
                             'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
                             'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
                             'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions(MtCO2e)': 'total_ghg',
                             'Direct GHG Emissions(MtCO2e)': 'direct_ghg',
                             'Indirect GHG Emissions(MtCO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)

filtered2013.head()

,bbl,strt_num,strt_name,borough,site_eui,wnsite_eui,src_eui,wnsrc_eui,energ_star,total_ghg,direct_ghg,indrct_ghg,sq_footage,prop_type,t_ghg_psqf
0,1000410001,NaN,NaN,Manhattan,nan,nan,nan,nan,nan,nan,nan,nan,918919.00000,Office,nan
1,4021460028,63-10,108 STREET,Queens,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan
2,2027620097,NaN,NaN,Bronx,nan,nan,nan,nan,nan,nan,nan,nan,64200.00000,Multifamily Housing,nan
3,1006700001,201,11 AVENUE,Manhattan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan
4,2048300034,NaN,NaN,Bronx,nan,nan,nan,nan,nan,nan,nan,nan,50000.00000,Multifamily Housing,nan


In [10]:
data2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14112 entries, 0 to 14111
Data columns (total 23 columns):
BBL                                                    14112 non-null int64
Street Number                                          11416 non-null float64
Street Name                                            14106 non-null object
Borough                                                14112 non-null object
Postcode                                               14112 non-null int64
Benchmarking Submission                                14112 non-null object
Entry Number                                           11989 non-null float64
Site EUI(kBtu/ft2)                                     11074 non-null float64
Weather Normalized Source EUI(kBtu/ft2)                11071 non-null float64
Indoor Water Intensity (All Water Sources)(gal/ft2)    6867 non-null float64
Reported Water Method                                  11989 non-null object
ENERGY STAR Score                         

In [295]:
filtered2013.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16014 entries, 0 to 16169
Data columns (total 15 columns):
bbl           16014 non-null int64
strt_num      14423 non-null object
strt_name     14458 non-null object
borough       16014 non-null object
site_eui      11467 non-null float64
wnsite_eui    9745 non-null float64
src_eui       11467 non-null float64
wnsrc_eui     9745 non-null float64
energ_star    2002 non-null float64
total_ghg     11603 non-null float64
direct_ghg    11926 non-null float64
indrct_ghg    12103 non-null float64
sq_footage    12906 non-null float64
prop_type     12977 non-null object
t_ghg_psqf    11546 non-null float64
dtypes: float64(10), int64(1), object(4)
memory usage: 2.0+ MB


In [296]:
# In 2012, our BBL data is already a non-string object so we don't need to do all the transformations
# we did for 2017 and 2018
filtered2012 = data2012[data2012['BBL'].notnull()]
filtered2012.head()

,BBL,Street Number,Street Name,Borough,Postcode,Benchmarking Submission,Entry Number,Site EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),Indoor Water Intensity (All Water Sources)(gal/ft2),Reported Water Method,ENERGY STAR Score,Total GHG Emissions(MtCO2e),Property Floor Area (Buildngs and Parking)(ft2),Primary Property Type - Self Selected,Number of Buildings,BIN,Latitude,Longitude,Community Board,Council District,Census Tract,NTA
0,1000010010,1.00000,GOVERNORS ISLAND,MANHATTAN,10004,No Record as of 08/1/13,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,1,NaN,40.69010,-74.01287,1.00000,1.00000,5.00000,park-cemetery-etc-Manhattan ...
1,1000020002,nan,MARGINAL STREET,MANHATTAN,10004,No Record as of 08/1/13,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,0,NaN,nan,nan,nan,nan,nan,NaN
2,1000047501,1.00000,WATER STREET,MANHATTAN,10004,Yes,41.00000,102.00000,287.90000,16.37000,Manual,75,25932.68000,2428325.00000,Office,1,1000005,40.70244,-74.01274,1.00000,1.00000,9.00000,Battery Park City-Lower Manhattan ...
3,1000057501,125.00000,BROAD STREET,MANHATTAN,10004,Yes,3871.00000,119.60000,261.50000,nan,Manual,70,11637.42000,1338000.00000,Office,1,1000006,40.70211,-74.01125,1.00000,1.00000,9.00000,Battery Park City-Lower Manhattan ...
4,1003620001,134.00000,AVENUE D,MANHATTAN,10009,Yes,8312.00000,30.30000,85.70000,1.57000,ABS,NaN,13709.99000,5207812.00000,Multifamily Housing,7,1077504;1077538;1077539;1077540;1077541;107754...,40.72445,-73.97546,3.00000,2.00000,24.00000,Lower East Side ...


In [297]:
# drop columns we won't visualize to reduce file size
filtered2012.drop(['BIN', 'Postcode', 'Benchmarking Submission', 'Entry Number', 'Indoor Water Intensity (All Water Sources)(gal/ft2)', 'Reported Water Method', 'Number of Buildings', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'NTA'], axis = 1, inplace = True)
filtered2012.head()


,BBL,Street Number,Street Name,Borough,Site EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Property Floor Area (Buildngs and Parking)(ft2),Primary Property Type - Self Selected
0,1000010010,1.00000,GOVERNORS ISLAND,MANHATTAN,nan,nan,NaN,nan,nan,NaN
1,1000020002,nan,MARGINAL STREET,MANHATTAN,nan,nan,NaN,nan,nan,NaN
2,1000047501,1.00000,WATER STREET,MANHATTAN,102.00000,287.90000,75,25932.68000,2428325.00000,Office
3,1000057501,125.00000,BROAD STREET,MANHATTAN,119.60000,261.50000,70,11637.42000,1338000.00000,Office
4,1003620001,134.00000,AVENUE D,MANHATTAN,30.30000,85.70000,NaN,13709.99000,5207812.00000,Multifamily Housing


In [298]:
# pd.to_numeric(arg, errors=’raise’, downcast=None)
# convert columns to floats or ints if that's what they contain
# filtered2012['Year Built'].value_counts()
cols_to_number = ['Property Floor Area (Buildngs and Parking)(ft2)', 'ENERGY STAR Score', 'Weather Normalized Source EUI(kBtu/ft2)', 'Weather Normalized Source EUI(kBtu/ft2)', 'Site EUI(kBtu/ft2)', 'Total GHG Emissions(MtCO2e)' ]
filtered2012[cols_to_number] = filtered2012[cols_to_number].apply(pd.to_numeric, errors='coerce')
filtered2012['BBL'] = filtered2012['BBL'].astype(int)
filtered2012.describe()

,BBL,Street Number,Site EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Property Floor Area (Buildngs and Parking)(ft2)
count,14112.00000,11416.00000,11074.00000,11071.00000,1864.00000,11024.00000,11989.00000
mean,2240990821.99943,1036.57113,441.91851,624.57568,62.75268,10700.46941,203823.26174
std,1239132878.67308,1657.14912,6792.63590,8420.44874,29.28362,269303.94263,478012.56380
min,1000010010.00000,1.00000,0.00000,0.00000,1.00000,0.12000,0.00000
25%,1013205637.25000,152.00000,59.10000,100.80000,43.00000,330.02750,66000.00000
50%,2032150041.00000,461.00000,76.70000,128.80000,71.00000,521.39000,95573.00000
75%,3067525050.50000,1381.25000,97.30000,170.70000,87.00000,1006.45000,169055.00000
max,5080080134.00000,76179.00000,494449.30000,576072.00000,100.00000,22449210.27000,14217119.00000


In [299]:
# compute emissions per sq ft based on sq ft / total ghg emissions
filtered2012['Emissions Per Sq Ft'] = np.where(filtered2012['Property Floor Area (Buildngs and Parking)(ft2)'] <= 0, 0, filtered2012['Total GHG Emissions(MtCO2e)'] /filtered2012['Property Floor Area (Buildngs and Parking)(ft2)'])
filtered2012.describe()

,BBL,Street Number,Site EUI(kBtu/ft2),Weather Normalized Source EUI(kBtu/ft2),ENERGY STAR Score,Total GHG Emissions(MtCO2e),Property Floor Area (Buildngs and Parking)(ft2),Emissions Per Sq Ft
count,14112.00000,11416.00000,11074.00000,11071.00000,1864.00000,11024.00000,11989.00000,11037.00000
mean,2240990821.99943,1036.57113,441.91851,624.57568,62.75268,10700.46941,203823.26174,0.03434
std,1239132878.67308,1657.14912,6792.63590,8420.44874,29.28362,269303.94263,478012.56380,0.53577
min,1000010010.00000,1.00000,0.00000,0.00000,1.00000,0.12000,0.00000,0.00000
25%,1013205637.25000,152.00000,59.10000,100.80000,43.00000,330.02750,66000.00000,0.00418
50%,2032150041.00000,461.00000,76.70000,128.80000,71.00000,521.39000,95573.00000,0.00559
75%,3067525050.50000,1381.25000,97.30000,170.70000,87.00000,1006.45000,169055.00000,0.00729
max,5080080134.00000,76179.00000,494449.30000,576072.00000,100.00000,22449210.27000,14217119.00000,39.04864


In [302]:
# rename some of the columns to compare datasets across years more easily
filtered2012.rename(columns={
#                              'Order': 'order',
#                              'Property Name': 'prop_name',
#                              'Address 1 (self-reported)': 'address',
                             'Street Number': 'strt_num',
                             'Street Name': 'strt_name',
                             'Borough': 'borough',
                             'Property Floor Area (Buildngs and Parking)(ft2)': 'sq_footage', 
                             'Primary Property Type - Self Selected': 'prop_type',
#                              'Year Built': 'year_built',
                             'ENERGY STAR Score': 'energ_star',
#                              'Source EUI(kBtu/ft2)': 'src_eui',
                             'Weather Normalized Source EUI(kBtu/ft2)': 'wnsrc_eui',
                             'Site EUI(kBtu/ft2)': 'site_eui',
#                              'Weather Normalized Site EUI(kBtu/ft2)': 'wnsite_eui',
#                              'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'wn_sei_kwh',
#                              'Electricity Use - Grid Purchase (kBtu)': 'eu_gp_kbtu',
#                              'Electricity Use - Grid Purchase (kWh)': 'eu_gp_kwh',
#                              'Weather Normalized Site Electricity (kWh)': 'wnse_kwh',
                             'Total GHG Emissions(MtCO2e)': 'total_ghg',
                             'Direct GHG Emissions(MtCO2e)': 'direct_ghg',
                             'Indirect GHG Emissions(MtCO2e)': 'indrct_ghg',
                             'BBL': 'bbl',
                             'Emissions Per Sq Ft': 't_ghg_psqf'
                            }, inplace=True)

filtered2012.head()

,bbl,strt_num,strt_name,borough,site_eui,wnsrc_eui,energ_star,total_ghg,sq_footage,prop_type,t_ghg_psqf
0,1000010010,1.00000,GOVERNORS ISLAND,MANHATTAN,nan,nan,nan,nan,nan,NaN,nan
1,1000020002,nan,MARGINAL STREET,MANHATTAN,nan,nan,nan,nan,nan,NaN,nan
2,1000047501,1.00000,WATER STREET,MANHATTAN,102.00000,287.90000,75.00000,25932.68000,2428325.00000,Office,0.01068
3,1000057501,125.00000,BROAD STREET,MANHATTAN,119.60000,261.50000,70.00000,11637.42000,1338000.00000,Office,0.00870
4,1003620001,134.00000,AVENUE D,MANHATTAN,30.30000,85.70000,nan,13709.99000,5207812.00000,Multifamily Housing,0.00263


In [304]:
filtered2012.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14112 entries, 0 to 14111
Data columns (total 11 columns):
bbl           14112 non-null int64
strt_num      11416 non-null float64
strt_name     14106 non-null object
borough       14112 non-null object
site_eui      11074 non-null float64
wnsrc_eui     11071 non-null float64
energ_star    1864 non-null float64
total_ghg     11024 non-null float64
sq_footage    11989 non-null float64
prop_type     11953 non-null object
t_ghg_psqf    11037 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 1.3+ MB


In [11]:
data2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4081 entries, 0 to 4080
Data columns (total 23 columns):
BBL                                 4081 non-null int64
Street Number                       4033 non-null object
Street Name                         4081 non-null object
Borough                             4081 non-null object
Postcode                            4081 non-null int64
Benchmarking Submission             4081 non-null object
Entry Number                        2546 non-null float64
Site EUI                            2319 non-null float64
Weather Normalized Source EUI       2319 non-null float64
Water per Square Foot               385 non-null float64
ENERGY STAR Score                   1593 non-null float64
GHG                                 2309 non-null float64
Reported Building Square Footage    2423 non-null float64
Reported Facility Type              2535 non-null object
Number of Buildings                 2546 non-null float64
Reported BINs                    

In [305]:
# write the cleaned dataframes to new csv files
filtered2018.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018_formatted.csv")
filtered2017.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_formatted.csv")
filtered2016.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_formatted.csv")
filtered2015.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_formatted.csv")
filtered2014.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_formatted.csv")
filtered2013.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_formatted.csv")
filtered2012.to_csv("data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_formatted.csv")



